In [33]:
from transformers import BertForSequenceClassification, BertTokenizer
from transformers import MarianMTModel, MarianTokenizer
import time
from sklearn.metrics import accuracy_score
import pandas as pd
from transformers import Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from torch.utils.data import Dataset
from transformers import EarlyStoppingCallback

model_name = "Helsinki-NLP/opus-mt-vi-en"
tokenizer_translate = MarianTokenizer.from_pretrained(model_name)
model_translate = MarianMTModel.from_pretrained(model_name)

# Load model and tokenizer
model = BertForSequenceClassification.from_pretrained("/kaggle/input/107-spensy-ai/pytorch/default/1")
tokenizer = BertTokenizer.from_pretrained("/kaggle/input/107-spensy-ai/pytorch/default/1")

/opt/conda/lib/python3.10/site-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
# Container for misclassified examples
misclassified_data = []

# Define dataset class
class SentimentDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [35]:
def translate_vi_to_en(text):
    # Làm sạch và chuẩn hóa văn bản nếu cần
    text = text.strip()
    inputs = tokenizer_translate(text, return_tensors="pt", truncation=True)
    translated = model_translate.generate(**inputs)
    translated_text = tokenizer_translate.decode(translated[0], skip_special_tokens=True)
    return translated_text

In [43]:
import torch

def classify_text(translated_text):
    inputs = tokenizer(translated_text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    prediction = torch.argmax(outputs.logits, dim=1).item()
    return "Positive" if prediction == 1 else "Negative"

In [51]:
new_text = input("Nhập câu bình luận vào đây: ")

# Translation timing
start_translate = time.time()
translated_text = translate_vi_to_en(new_text)
end_translate = time.time()
print("Translated Text:", translated_text)
print("Translation Time:", end_translate - start_translate, "seconds")

# Classification timing
start_classify = time.time()
prediction = classify_text(translated_text)
end_classify = time.time()
print("Kết quả dự đoán:", prediction)
print("Thời gian thực thi:", end_classify - start_classify, "seconds")

# Accuracy feedback
true_label = input("Cho tôi giá trị thực sự (Positive/Negative): ").strip().capitalize()
if true_label in ["Positive", "Negative"]:
    accuracy = accuracy_score([true_label], [prediction])
    print("Độ chính xác:", accuracy)
    if accuracy == 0:
        misclassified_data.append({'text': translated_text, 'label': 1 if true_label == "Positive" else 0})
        print("Cảm ơn bạn đã feedback chúng tôi sẽ huấn luyện lại mô hình cho tốt hơn.")
else:
    print("Invalid label input.")

Nhập câu bình luận vào đây:  Thiệt là nhiều khi muốn làm Taowork cho khỏe Kêu làm nội dung thì tìm trên gg xong copy y chang lại rồi gửi, không thêm bớt, không sửa font hay chính tả gì cả Kêu làm word với ppt thì làm cho có, ko căng chỉnh font, lề gì. Ppt thì để 1 đống chữ ko biết tự lược lại nội dung cho ngắn gọn, sai chính tả lum la, kêu chỉnh cả trăm lần cuối cùng đứa đi nhắc phải là đứa tự chỉnh Kêu thuyết thì thì ngại, ko muốn. Có lên thì ppt có gì đọc y chang, không xem trước bài ở nhà. Đọc cho có rồi về Nhiều khi muốn xin giảng viên làm bài 1 mình cho rồi nhưng gv ko cho chứ làm việc nhóm mà gặp đủ 3 thể loại như trên, t mà là nhóm trưởng t trừ điểm hết khỏi ý kiến với t


Translated Text: A lot of times you want to make Taowork healthy. If you want to do the content, you can do the same copy on your gg and then you can send, no less, no more font or spelling. If you want to do it with ppt, then you can do it.
Translation Time: 4.5567896366119385 seconds
Kết quả dự đoán: Positive
Thời gian thực thi: 0.1603682041168213 seconds


Cho tôi giá trị thực sự (Positive/Negative):  negative


Độ chính xác: 0.0
Cảm ơn bạn đã feedback chúng tôi sẽ huấn luyện lại mô hình cho tốt hơn.


In [38]:
def classify_and_get_feedback(text):
    translated_text = translate_vi_to_en(text)
    print("Translated Text:", translated_text)

    start_classify = time.time()
    predicted_label = classify_text(translated_text)
    end_classify = time.time()
    print("Predicted Label:", predicted_label)
    print("Classification Time:", end_classify - start_classify, "seconds")

    user_label = input("If incorrect, please enter the correct label (Positive/Negative), else press Enter: ")
    if user_label and user_label != predicted_label:
        misclassified_data.append({'text': translated_text, 'label': 1 if user_label == "Positive" else 0})
        print("Feedback recorded. Model will learn from this.")

In [50]:
def retrain_with_feedback(train_dataset):
    if misclassified_data:
        feedback_df = pd.DataFrame(misclassified_data)
        feedback_encodings = tokenizer(feedback_df['text'].tolist(), truncation=True, padding=True)
        feedback_labels = feedback_df['label'].tolist()

        feedback_dataset = SentimentDataset(feedback_encodings, feedback_labels)
        combined_train_dataset = torch.utils.data.ConcatDataset([train_dataset, feedback_dataset])

        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=1,  
            per_device_train_batch_size=8,
            evaluation_strategy="no",
            save_steps=500,
            logging_dir='./logs',
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=combined_train_dataset,
            callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
        )

        # Retrain model
        trainer.train()
        print("Model retrained with user feedback.")
        misclassified_data.clear()  

while True:
    new_text = input("Enter a sentence for classification (or 'quit' to exit): ")
    if new_text.lower() == 'quit':
        break
    classify_and_get_feedback(new_text)

    # Huấn luyện lại nếu dữ liệu >=5 
    if len(misclassified_data) >= 5:  # Adjust threshold as needed
        retrain_with_feedback(train_dataset)

Enter a sentence for classification (or 'quit' to exit):  Thiệt là nhiều khi muốn làm Taowork cho khỏe Kêu làm nội dung thì tìm trên gg xong copy y chang lại rồi gửi, không thêm bớt, không sửa font hay chính tả gì cả Kêu làm word với ppt thì làm cho có, ko căng chỉnh font, lề gì. Ppt thì để 1 đống chữ ko biết tự lược lại nội dung cho ngắn gọn, sai chính tả lum la, kêu chỉnh cả trăm lần cuối cùng đứa đi nhắc phải là đứa tự chỉnh Kêu thuyết thì thì ngại, ko muốn. Có lên thì ppt có gì đọc y chang, không xem trước bài ở nhà. Đọc cho có rồi về Nhiều khi muốn xin giảng viên làm bài 1 mình cho rồi nhưng gv ko cho chứ làm việc nhóm mà gặp đủ 3 thể loại như trên, t mà là nhóm trưởng t trừ điểm hết khỏi ý kiến với t


Translated Text: A lot of times you want to make Taowork healthy. If you want to do the content, you can do the same copy on your gg and then you can send, no less, no more font or spelling. If you want to do it with ppt, then you can do it.
Predicted Label: Positive
Classification Time: 0.15396928787231445 seconds


If incorrect, please enter the correct label (Positive/Negative), else press Enter:  negative


Feedback recorded. Model will learn from this.


Enter a sentence for classification (or 'quit' to exit):  quit
